# Reddit data
 > Download from [here](https://amazon-reviews-2023.github.io/)

In [13]:
import os
import praw
from dotenv import load_dotenv
from pprint import pprint

In [14]:
load_dotenv()

reddit = praw.Reddit(
    client_id=os.getenv("REDDIT_CLIENT_ID"),
    client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
    user_agent="get_reddit_posts",
)

In [15]:
submissions = [s for s in reddit.subreddit("auckland").top(time_filter="week", limit=100)]

In [17]:
example = submissions[2]

pprint(vars(example))

{'_additional_fetch_params': {},
 '_comments_by_id': {},
 '_fetched': False,
 '_reddit': <praw.reddit.Reddit object at 0x000002A8E2024E60>,
 'all_awardings': [],
 'allow_live_comments': False,
 'approved_at_utc': None,
 'approved_by': None,
 'archived': False,
 'author': Redditor(name='Gypsyfella'),
 'author_flair_background_color': None,
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_template_id': None,
 'author_flair_text': None,
 'author_flair_text_color': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_qfcbu454',
 'author_is_blocked': False,
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'banned_at_utc': None,
 'banned_by': None,
 'can_gild': False,
 'can_mod_post': False,
 'category': None,
 'clicked': False,
 'comment_limit': 2048,
 'comment_sort': 'confidence',
 'content_categories': None,
 'contest_mode': False,
 'created': 1734224242.0,
 'created_utc': 1734224242.0,
 'discussion_type': None,
 'distinguishe

# Create Chroma DB index

In [24]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DataFrameLoader
from langchain_chroma import Chroma
from tqdm.auto import tqdm

In [26]:
dimensions = 1536

embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=dimensions)

vector_store = Chroma(
    collection_name="amazon_reviews",
    embedding_function=embeddings,
    persist_directory="../data/chroma",
)

In [27]:
loader = DataFrameLoader(df, page_content_column="text")
docs = loader.load()

# add documents in batches
batch_size = 1000
for i in tqdm(range(0, len(docs), batch_size)):
    vector_store.add_documents(documents=docs[i : i + batch_size])

  0%|          | 0/7 [00:00<?, ?it/s]

In [36]:
def get_results_df(results):
    data = []
    for doc, score in results:
        row = doc.metadata
        row["page_content"] = doc.page_content
        row["score"] = score
        data.append(row)

    # Create dataframe
    df = pd.DataFrame(data)

    return df

In [37]:
results = vector_store.similarity_search_with_score("I like the controller", k=5)

get_results_df(results)

,date,helpful_vote,product,rating,title,page_content,score
0,2021-02-25,0,PlayStation 5 Console,5,Like everything except 1 thing,I like everything about it. Except the control...,0.459350
1,2022-09-23,0,Xbox Series S,5,Great product,The way the controller feels.,0.690670
2,2021-06-25,0,Xbox Series S,5,controller,good but the controller was used,0.709363
3,2021-04-30,0,PlayStation 5 Console,5,Best console,"Its great so far, they nailed it with the new ...",0.729130
4,2021-02-16,1,PlayStation 5 Console,5,Best Gaming System,Finally got it!!!! This thing is awesome. The ...,0.749155
